### Practice Problem 5: RCH Industries' Differentiation

In [1]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver('factory', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [2]:
stylish_p_to_xd_transportation_cost_coef = [[30,50], [23,66], [35,14], [70,12], [65,70]]
leisure_p_to_xd_transportation_cost_coef = [[33,55], [25,73], [39,15], [77,13], [12,14]]

stylish_xd_to_dc_transportation_cost_coef = [[12,25,22,40,41], [65,22,23,12,15]]
leisure_xd_to_dc_transportation_cost_coef = [[13,28,24,44,45], [72,24,25,13,17]]

stylish_demand = [130,45,70,100,5]
leisure_demand = [15,45,40,100,175]

max_stylish_demand = sum(stylish_demand)
max_leisure_demand = sum(leisure_demand)

plant_capacity_stylish = [150, 300, 90, 140, 220]
plant_capacity_leisure = [200, 300, 70,  30,  220]
plant_capacity_combined = [200, 300, 666, 666, 666]

n_plants = len(plant_capacity_combined)
n_xds = len(stylish_p_to_xd_transportation_cost_coef[0])
n_dcs = len(stylish_demand)

print(n_plants, 'plants')
print(n_xds, 'xds')
print(n_dcs, 'dcs')

5 plants
2 xds
5 dcs


In [6]:
F_stylish_p_to_xd = {}
for p in range(n_plants):
    for xd in range(n_xds):
        F_stylish_p_to_xd[(p,xd)] = solver.IntVar(0, max_stylish_demand,
                                                 'f_stl_p{p}_xd{xd}'.format(p=p,xd=xd))
        
F_leisure_p_to_xd = {}
for p in range(n_plants):
    for xd in range(n_xds):
        F_leisure_p_to_xd[(p,xd)] = solver.IntVar(0, max_leisure_demand,
                                                 'f_lsr_p{p}_xd{xd}'.format(p=p,xd=xd))
        
F_stylish_xd_to_dc = {}
for xd in range(n_xds):
    for dc in range(n_dcs):
        F_stylish_xd_to_dc[(xd,dc)] = solver.IntVar(0, max_stylish_demand,
                                                 'f_stl_xd{xd}_dc{dc}'.format(dc=dc,xd=xd))
        
F_leisure_xd_to_dc = {}
for xd in range(n_xds):
    for dc in range(n_dcs):
        F_leisure_xd_to_dc[(xd,dc)] = solver.IntVar(0, max_leisure_demand,
                                                 'f_lsr_xd{xd}_dc{dc}'.format(dc=dc,xd=xd))


In [ ]:
### production costs 

# THERE AREN'T ANY


In [10]:
#### transportation costs #####

## from Plants to XDs
# stylish 
stylish_p_to_xd_transportation_cost = [stylish_p_to_xd_transportation_cost_coef[p][xd]*\
                                       F_stylish_p_to_xd[(p, xd)]\
                                       for p in range(n_plants)\
                                       for xd in range(n_xds)]

# leisure
leisure_p_to_xd_transportation_cost = [leisure_p_to_xd_transportation_cost_coef[p][xd]*\
                                       F_leisure_p_to_xd[(p, xd)]\
                                       for p in range(n_plants)\
                                       for xd in range(n_xds)]

## from XDs to DCs
# stylish 
stylish_xd_to_dc_transportation_cost = [stylish_xd_to_dc_transportation_cost_coef[xd][dc]*\
                                       F_stylish_xd_to_dc[(xd, dc)]\
                                       for xd in range(n_xds)\
                                       for dc in range(n_dcs)]

# leisure
leisure_xd_to_dc_transportation_cost = [leisure_xd_to_dc_transportation_cost_coef[xd][dc]*\
                                       F_leisure_xd_to_dc[(xd, dc)]\
                                       for xd in range(n_xds)\
                                       for dc in range(n_dcs)]



In [11]:
# capacity Constraints

# flow out of the factory cannot exceed capacity
# stylish
for p in range(n_plants):
    solver.Add(solver.Sum([F_stylish_p_to_xd[(p,xd)] for xd in range(n_xds)])\
               <= plant_capacity_stylish[p])

# leisure
for p in range(n_plants):
    solver.Add(solver.Sum([F_leisure_p_to_xd[(p,xd)] for xd in range(n_xds)])\
               <= plant_capacity_leisure[p])

# combined
for p in range(n_plants):
    solver.Add(solver.Sum([F_stylish_p_to_xd[(p,xd)]+F_leisure_p_to_xd[(p,xd)]\
                           for xd in range(n_xds)])\
               <= plant_capacity_combined[p])
    
    


In [ ]:
### Demand constraints ###

#the flow into DCs must match their demands 
#stylish 
for dc in range(n_dcs):
    solver.Add(solver.Sum([F_stylish_xd_to_dc[(xd, dc)]\
                for xd in range(n_xds)]) == stylish_demand[dc])
    
#leisure 
for dc in range(n_dcs):
    solver.Add(solver.Sum([F_leisure_xd_to_dc[(xd, dc)]\
                for xd in range(n_xds)]) == leisure_demand[dc])

In [ ]:
## balance of flow constraints 
# what comes in to a XD must leave the DC

for xd in range(n_xds):
    

In [12]:
### total costs ###

total_cost = solver.Sum([solver.Sum(stylish_p_to_xd_transportation_cost),
                         solver.Sum(leisure_p_to_xd_transportation_cost),
                         solver.Sum(stylish_xd_to_dc_transportation_cost),
                         solver.Sum(leisure_xd_to_dc_transportation_cost)])
                         